<h1>NER测试<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#分词测试" data-toc-modified-id="分词测试-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>分词测试</a></span></li><li><span><a href="#切词之后使用NER模型" data-toc-modified-id="切词之后使用NER模型-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>切词之后使用NER模型</a></span></li></ul></div>

获得`cls_features`需要使用能适应变长输入的预训练模型, 或者自己训练2048长度以上的定长模型. 

# 分词测试

In [1]:
use_proxy = False
proxies={"http_proxy": "127.0.0.1:10809",
        "https_proxy": "127.0.0.1:10809"} if use_proxy else None

In [2]:
max_sen_in_train='''(" The family first appeared in 2006 and the family have been involved in a number of the show's most high-profile storylines, most notably John Paul McQueen's (James Sutton) affair with Craig Dean (Guy Burnet); Jacqui McQueen's (Claire Cooper) whirlwind relationship with Tony Hutchinson (Nick Pickard); Myra McQueen's (Nicole Barber-Lane) long-lost son Niall Rafferty's (Barry Sloane) revenge on his family by holding them hostage in an abandoned church and blowing it up, ultimately killing his half-sister Tina Reilly (Leah Hackett); Theresa McQueen's (Jorgie Porter) pregnancy by her cousin Carmel McQueen's (Gemma Merna) fiancé Calvin Valentine (Ricky Whittle) and later shooting him dead on their wedding day; Mercedes McQueen's (Jennifer Metcalfe) affair with her fiancé Riley Costello's (Rob Norbuy) father Carl (Paul Opacic); being kidnapped by Riley's grandfather Silas; staking Riley's second cousin Mitzee Minniver; Jacqui coping with the death of her husband Rhys Ashworth (Andrew Moss) in a bus crash, learning that he had been having an affair with Cindy Cunningham (Stephanie Waring) and that he got Sinead O'Connor (Stephanie Davis) pregnant; Mercedes stalking Mitzeee (Rachel Shenton) and stabbing herself and framing her; Carmel's facial disfigurement; Myra faking her own death to escape her daughter Mercedes' evil husband, Dr. Paul Browning (Joseph Thompson); Mercedes killing her husband Doctor Browning by striking him over the head with a shovel; John Paul's male rape at the hands of his pupil Finn O'Connor (Keith Rice); the train crash which ultimately killed Carmel; Mercedes faking her death to help Grace Black (Tamara Wall) get revenge on Freddie Roscoe (Charlie Clapham); Theresa donating her kidney to Nico Blake (Persephone Swales-Dawson); Porsche (Twinnie Lee Moore) and Cleo McQueen's (Nadine Rose Mulkerrin) sexual abuse at the hands of their mother Reenie McQueen's (Zöe Lucker) fiancé Pete Buchanan (Kai Owen); Phoebe McQueen's (Mandip Gill) murder in hospital by the Gloved Hand Killer; the stillbirth of Mercedes' baby Gabriel McQueen; John Paul's transgender boss Sally St. Claire (Annie Wallace) being revealed as his biological father, Mercedes being framed for drugs by Joanne Cardsley (Rachel Leskovac), Celine McQueen (Sarah George) and Diego Salvador Martinez Hernandez De La Cruz (Juan Pablo Yepez)'s sham wedding for money and Celine being murdered by her ex-boyfriend and serial killer Cameron Campbell (Cameron Moore) after discover he causes the fire at the fair on Halloween 2016.", 2534)'''


In [3]:
from transformers import AutoConfig, AutoModel, BertConfig, AutoTokenizer

In [4]:
model_name = 'xlnet-base-cased'

tokenizer_XLNET = AutoTokenizer.from_pretrained(model_name,proxies=proxies)
# model_XLNET = AutoModel.from_config(config)

In [5]:
test = tokenizer_XLNET.encode('hello i am jim','who are you',add_special_tokens=True)
test = tokenizer_XLNET.decode(test)
test

'hello i am jim<sep> who are you<sep><cls>'

In [6]:
model_name2 = 'bert-base-uncased'

tokenizer_BERT = AutoTokenizer.from_pretrained(model_name2,proxies=proxies)
# model_XLNET = AutoModel.from_config(config)


In [7]:
test1 = tokenizer_XLNET.tokenize(max_sen_in_train)
len(test1)

693

In [8]:
test2 = tokenizer_BERT.tokenize(max_sen_in_train)
len(test2)

595

# 切词之后使用NER模型

In [9]:
import spacy
spacy.prefer_gpu()
# spacy.prefer_cpu()
nlp = spacy.load("en_core_web_lg")

In [10]:
res_nlp = nlp(max_sen_in_train)
tokens =  [str(i) for i in res_nlp.doc]
len(tokens)

508

In [11]:
tokens_XLNET = tokenizer_XLNET.encode(tokens)
len(tokens_XLNET)

510

`encode_plus`返回: `dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])`

In [12]:
# 原始句子
model_input1 = tokenizer_XLNET.encode_plus(max_sen_in_train,
                                          add_special_tokens=True,
                                         return_tensors='pt')
model_input1['input_ids'].shape

torch.Size([1, 695])

In [13]:
# 分词后
model_input2 = tokenizer_XLNET.encode_plus(tokens,
                                          add_special_tokens=True,
                                         return_tensors='pt')
model_input2['input_ids'].shape

torch.Size([1, 510])

**不能将分词后的tokens传入XLNET!**

In [14]:
model_input1['input_ids']

tensor([[   17,    10,    12,    32,   273,    89,  1144,    25,  1267,    21,
            18,   273,    47,    72,   899,    25,    24,   243,    20,    18,
           351,    26,    23,   127,   227,    13,  9697, 17411,    23,    19,
           127,  7003,   408,  1209,  2956, 21905,    26,    23,    17,    10,
         24500, 19435,    11,  7119,    33,  7454,  5153,    17,    10,  8696,
           117,  3571,  1942,    11,    97,   586, 25075,  2956, 21905,    26,
            23,    17,    10, 10294,  6121,  8101,    11, 31524,  1498,    33,
          3342, 27280,    17,    10,   353,  4287,  9117,  1896,    11,    97,
           631,   840,  2956, 21905,    26,    23,    17,    10,   353,   556,
          5457, 20213,    13,  3150,   667,    11,   206,    13,  5546,    46,
           966,  2971,  2225,  2942,  8037,   982,    26,    23,    17,    10,
         10760,   844, 20861,    93,    11,  9621,    31,    45,   273,    37,
          1757,   107,  8840,    25,    48,  4832,  

In [15]:
model_input2['input_ids']

tensor([[   10,    12,   152, 13606,  9447,     0,   153, 12153,   443,   305,
         13606,  9613,     0,     0,   153,   101, 13920,  1168,   305,  8676,
             0,  6296,  3937,    13,  9697,     0,    19,  6296,     0, 10089,
         17323,     0,     0,    10, 24500,     0,    11,     0,  3263,     0,
             0,    10,     0,     0,    11,    97,     0,     0,     0,    10,
             0,     0,    11,     0,     0,  3263,     0,     0,    10,     0,
             0,    11,    97,     0,     0,     0,    10,     0,     0,    13,
             0,    11,  2410,    13,     0,   672,     0,     0,     0,    10,
             0,     0,    11,     0,   253,  7496, 13606,  1164, 13551,     0,
             0,   153,   262,     0, 26329,   443,     0,   669,   499,    19,
             0,     0,  7496,  5960,    13,     0,     0, 18037,    10,     0,
             0,    11,    97,     0,     0,     0,    10,     0,     0,    11,
             0,  1164,  1921,     0,     0,     0,  